In [4]:
import os
import random
import pandas as pd
from typing import TypedDict, List, Literal
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from dotenv import load_dotenv
import re
import requests
from bs4 import BeautifulSoup



load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Configuración inicial
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatOpenAI(model="gpt-4o", temperature=0)

# Generar df con escenarios medievales
df_scenarios = pd.DataFrame({
    "scenario": ["Un joven noble se encuentra en un bosque mágico", 
                 "Un aventurero se encuentra en una cueva misteriosa", 
                 "Un aprendiz de mago se encuentra en una torre encantada"]
    
    
    
})



class AgentState(TypedDict):
    escenario: str
    heroe: str
    caramelo: str
    cantidad_puzzles: int
    max_puzzles: int
    puzzle: str
    respuesta: str
    puzzle_resuelto: bool
    story: List[str]
    adivinanzas: List[str]
    adivinanzas_resueltas: List[str]
    adivinanzas_no_resueltas: List[str]
    used_adivinanzas: List[str]
    puzzle_solution: str

GENERADOR_INTRO_PROMPT = """
Eres un narrador creativo. Crea una introducción cautivadora para una historia que incorpore:
1. Nombre del héroe: {hero_name}
2. Dulce favorito del héroe: {favorite_candy}
3. Escenario aleatorio: {random_scenario}

Tu introducción debe tener 3-5 oraciones, presentar al héroe, incorporar el dulce y establecer el escenario. 
Usa un lenguaje sencillo y amigable, apropiado para niños. Puedes usar emojis para hacer la historia más divertida.
"""

PROMPT_MAGO_BLANCO = """
Eres el Mago Blanco, un sabio guardián del bosque mágico. Tu tarea es poner a prueba al héroe {hero_name} con un acertijo o una pregunta matemática.

Contexto: El héroe debe responder correctamente para continuar su aventura en el bosque mágico.

Tienes dos opciones:
1. Usar una adivinanza de la siguiente lista: {adivinanzas}
2. Crear una pregunta matemática simple (suma, resta, multiplicación o división) relacionada con la historia del héroe.

Si eliges una adivinanza, preséntala tal como está.
Si creas una pregunta matemática, usa la herramienta de cálculo escribiendo [CALCULAR] seguido de la expresión entre corchetes. Que la pregunta sea de arimética simple, como
para niños de primaria.

Presenta tu acertijo o pregunta en este formato:
<acertijo>
Descripción: (Explica el desafío)
</acertijo>

<solucion>
Solución: (Proporciona la respuesta correcta)
</solucion>

Recuerda mantener un tono amigable y mágico, apropiado para una aventura en un bosque encantado.

"""
import random
import requests
from bs4 import BeautifulSoup
import PyPDF2
from io import BytesIO
import re




def obtener_adivinanzas_granada():
    url = "https://www.granada.org/inet/bibliote.nsf/63db9518fb82154cc125833e0039fbf7/0ddb42fc9ea3d9e1c125833d003ba713/$FILE/404%20-%20GYMKANA%20ADIVINANZAS%20DIBUJADAS.pdf"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    adivinanzas = []
    for p in soup.find_all('p'):
        if p.strong:
            pregunta = p.strong.text.strip()
            respuesta = p.text.split('Respuesta:')[-1].strip()
            adivinanzas.append({"pregunta": pregunta, "respuesta": respuesta})
    
    return adivinanzas

def obtener_adivinanzas_diezminutos():
    try:
        url = "https://www.diezminutos.es/maternidad/ninos/g42652853/adivinanzas-para-ninos-faciles/"
        response = requests.get(url)
        response.encoding = 'utf-8'  # Explicitly set the encoding
        soup = BeautifulSoup(response.text, 'html.parser')
        adivinanzas = []
        for div in soup.find_all('div', class_='body-text'):
            pregunta = div.find('strong')
            respuesta = div.find('em')
            if pregunta and respuesta:
                adivinanzas.append({
                    "pregunta": pregunta.text.strip().encode('utf-8').decode('utf-8', 'ignore'),
                    "respuesta": respuesta.text.strip().encode('utf-8').decode('utf-8', 'ignore')
                })
        return adivinanzas
    except Exception as e:
        print(f"Error al obtener adivinanzas de diezminutos: {e}")
        return []

def obtener_adivinanzas():
    fuentes = [obtener_adivinanzas_granada, obtener_adivinanzas_diezminutos]
    adivinanzas = []
    for fuente in fuentes:
        try:
            nuevas_adivinanzas = fuente()
            for adivinanza in nuevas_adivinanzas:
                adivinanza['pregunta'] = adivinanza['pregunta'].encode('utf-8', errors='ignore').decode('utf-8')
                adivinanza['respuesta'] = adivinanza['respuesta'].encode('utf-8', errors='ignore').decode('utf-8')
            adivinanzas.extend(nuevas_adivinanzas)
            if adivinanzas:
                print(f"Se obtuvieron {len(adivinanzas)} adivinanzas.")
                break
        except Exception as e:
            print(f"Error al obtener adivinanzas: {e}")
    
    if not adivinanzas:
        print("No se pudieron obtener adivinanzas. Usando adivinanza por defecto.")
        adivinanzas = [{"pregunta": "¿Qué tiene el rey en la panza?", "respuesta": "Ombligo"}]
    
    return adivinanzas
    


def intro_node(state: AgentState) -> AgentState:
    nombre_heroe = input("¿Cuál es el nombre de tu héroe? ")
    caramelo_favorito = input("¿Cuál es tu dulce favorito? ")
    random_scenario = df_scenarios["scenario"].sample().iloc[0]
    
    state['heroe'] = nombre_heroe
    state['caramelo'] = caramelo_favorito
    state['escenario'] = random_scenario
    
    formatted_prompt = GENERADOR_INTRO_PROMPT.format(
        hero_name=nombre_heroe,
        favorite_candy=caramelo_favorito,
        random_scenario=random_scenario
    )
    
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Genera la introducción de la historia.")
    ]
    response = model.invoke(messages)
    state['story'] = [response.content]
    print("\nHistoria generada:")
    print(state['story'][0])
    return state

def calcular(expresion):
    try:
        return eval(expresion)
    except:
        return "Error en el cálculo"
def mago_blanco_node(state: AgentState) -> AgentState:
    print("\nEl Mago Blanco está preparando un desafío...")
    try:
        adivinanzas = obtener_adivinanzas()
        used_adivinanzas = state.get('used_adivinanzas', [])
        available_adivinanzas = [a for a in adivinanzas if a not in used_adivinanzas]
        
        if not available_adivinanzas:
            available_adivinanzas = adivinanzas  # Reset if all have been used
        
        selected_adivinanza = random.choice(available_adivinanzas)
        used_adivinanzas.append(selected_adivinanza)
        state['used_adivinanzas'] = used_adivinanzas
        
        # Solo usamos la pregunta para el prompt del Mago Blanco
        adivinanza_prompt = f"Pregunta: {selected_adivinanza['pregunta']}"

        messages = [
            SystemMessage(content=PROMPT_MAGO_BLANCO.format(hero_name=state['heroe'], adivinanzas=adivinanza_prompt)),
            HumanMessage(content="Crea un desafío para este héroe basado en la adivinanza proporcionada.")
        ]
        response = model.invoke(messages)
        
        acertijo_match = re.search(r'<acertijo>(.*?)</acertijo>', response.content, re.DOTALL)
        solucion_match = re.search(r'<solucion>(.*?)</solucion>', response.content, re.DOTALL)
        
        if acertijo_match:
            state['puzzle'] = acertijo_match.group(1).strip()
            state['puzzle_solution'] = selected_adivinanza['respuesta']  # Usamos la respuesta original
        else:
            # Si no se pudo generar un acertijo, usamos la adivinanza seleccionada
            state['puzzle'] = selected_adivinanza['pregunta']
            state['puzzle_solution'] = selected_adivinanza['respuesta']
        
        print("\nDesafío del Mago Blanco:")
        print(state['puzzle'])
    except Exception as e:
        print(f"Error en mago_blanco_node: {e}")
        state['puzzle'] = "¿Qué tiene el rey en la panza?"
        state['puzzle_solution'] = "Ombligo"
    
    state['cantidad_puzzles'] = state.get('cantidad_puzzles', 0) + 1
    return state
        

def human_response_node(state: AgentState) -> AgentState:
    print(f"\nPregunta del Mago Blanco: {state['puzzle']}")
    state['respuesta'] = input("¿Cuál es tu respuesta al desafío del Mago Blanco? ")
    print(f"Tu respuesta: {state['respuesta']}")
    return state

def puzzle_evaluator_node(state: AgentState) -> AgentState:
    puzzle_solution = state.get('puzzle_solution', 'N/A')
    messages = [
        SystemMessage(content="Evalúa si la respuesta del héroe al desafío es correcta. Explica por qué es correcta o incorrecta. Si es incorrecta, proporciona la respuesta correcta."),
        HumanMessage(content=f"Pregunta: {state['puzzle']}\nSolución correcta: {puzzle_solution}\nRespuesta del héroe: {state['respuesta']}\n¿Es correcta?")
    ]
    response = model.invoke(messages)
    state['puzzle_resuelto'] = 'correcta' in response.content.lower()
    print(f"\nEl Mago Blanco dice: {response.content}")
    return state

def should_continue(state: AgentState) -> Literal["continuar", "terminar"]:
    if state.get('puzzle_resuelto', False):
        print("\n🎉 ¡Felicidades! Has superado el desafío del Mago Blanco.")
        return "terminar"
    elif state['cantidad_puzzles'] < state['max_puzzles']:
        print("\n🔄 El Mago Blanco te dará otra oportunidad.")
        return "continuar"
    else:
        print("\n⏰ Has agotado tus oportunidades con el Mago Blanco.")
        return "terminar"

# Creación del grafo
builder = StateGraph(AgentState)

# Agregar nodos
builder.add_node("presentacion_historia", intro_node)
builder.add_node("mago_blanco", mago_blanco_node)
builder.add_node("respuesta_humano", human_response_node)
builder.add_node("evaluador_puzzle", puzzle_evaluator_node)

# Definir bordes
builder.add_edge("presentacion_historia", "mago_blanco")
builder.add_edge("mago_blanco", "respuesta_humano")
builder.add_edge("respuesta_humano", "evaluador_puzzle")

# Borde condicional basado en la evaluación del desafío
builder.add_conditional_edges(
    "evaluador_puzzle",
    should_continue,
    {
        "continuar": "mago_blanco",
        "terminar": END
    }
)

# Establecer el punto de entrada
builder.set_entry_point("presentacion_historia")

# Compilar el grafo
graph = builder.compile()

# Ejecutar el grafo
state = {
    "max_puzzles": 3,
    "cantidad_puzzles": 0,
    "story": [],
    "puzzle_solution": "N/A",
    "used_adivinanzas": []
}


print("\n🧙‍♂️ ¡Bienvenido a la aventura en el bosque mágico! 🧙‍♂️")
try:
    final_state = graph.invoke(state)
    
    print("\n📜 Resumen final:")
    print(f"Historia: {final_state['story'][0]}")
    print(f"Desafíos enfrentados: {final_state['cantidad_puzzles']}")
    if final_state.get('puzzle_resuelto', False):
        print("🎊 El héroe ha superado los desafíos del Mago Blanco y continúa su aventura en el bosque mágico.")
    else:
        print("🔁 El héroe no ha superado todos los desafíos del Mago Blanco y debe buscar otro camino en el bosque.")
except Exception as e:
    print(f"Error durante la ejecución del juego: {e}")
    print("A pesar del error, la aventura continúa...")


🧙‍♂️ ¡Bienvenido a la aventura en el bosque mágico! 🧙‍♂️

Historia generada:
En lo alto de una torre encantada, donde las estrellas parecían susurrar secretos mágicos, vivía Fede, un aprendiz de mago con un corazón valiente y una sonrisa traviesa 😊. Fede siempre llevaba en su bolsillo una Tita, su dulce favorito, que le daba fuerzas en sus aventuras mágicas 🍫. Un día, mientras practicaba hechizos, descubrió un misterioso libro que brillaba con una luz dorada... ¡y así comenzó su gran aventura! ✨📖

El Mago Blanco está preparando un desafío...


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


No se pudieron obtener adivinanzas. Usando adivinanza por defecto.

Desafío del Mago Blanco:
Descripción: ¡Bienvenido, valiente héroe Fede! Para continuar tu aventura en el bosque mágico, debes resolver este enigma. Escucha con atención:

¿Qué tiene el rey en la panza?

Piensa bien antes de responder, pues solo con la respuesta correcta podrás seguir adelante en tu noble misión.

Pregunta del Mago Blanco: Descripción: ¡Bienvenido, valiente héroe Fede! Para continuar tu aventura en el bosque mágico, debes resolver este enigma. Escucha con atención:

¿Qué tiene el rey en la panza?

Piensa bien antes de responder, pues solo con la respuesta correcta podrás seguir adelante en tu noble misión.
Tu respuesta: Ombligo

El Mago Blanco dice: Sí, la respuesta del héroe es correcta. La pregunta "¿Qué tiene el rey en la panza?" es un enigma que juega con la idea de que, independientemente de su estatus o poder, todos los seres humanos tienen ciertas características físicas comunes. En este caso, la